In [1]:
import optuna
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.transforms import v2 as transforms

from utils import get_device, run_epochs, get_test_cifar10, get_train_cifar10

from models import ResNet18, DenseNet, PreActResNet

/homes/b24richa/Documents/EDL/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
rotation_degree = 45
translation = (0.2, 0.2) 
shearing = (10, 20, 0, 10)  # horizontal shear: 10–20°, vertical shear: 0–10°
kernel_size = 5

transformations = {
    # Geometric
    "rotation": transforms.RandomRotation(rotation_degree),
    "translation": transforms.RandomAffine(degrees=0, translate=translation),
    "shearing": transforms.RandomAffine(degrees=0, shear=shearing),

    # Non geometric
    "horizontal_flip": transforms.RandomHorizontalFlip(),
    "vertical_flip": transforms.RandomVerticalFlip(),
    "crop": transforms.RandomCrop(32, padding=4),
    "color_jitter": transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1),
    # "noise_injection": transforms.GaussianNoise(),
    "kernel": transforms.GaussianBlur(kernel_size=kernel_size),

    # Erasing
    "random_erasing": transforms.RandomErasing(),
}
optuna_range = {
    
}

In [8]:

def objective(trial):
    boolean_trials = {
        f"use_{key}": trial.suggest_categorical(f'use{key}', [True, False]) for key in transformations.keys()
    }
    transform_list = []
    
    for transform_name, transform in transformations.items():
        if boolean_trials[f"use_{transform_name}"]:
            transform_list.append(transform)

    transform_list.extend([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    transform = transforms.Compose(transform_list)
    
    model = ResNet18()
    device = get_device()
    model.to(device)
    
    hyperparams = {
        "criterion": nn.CrossEntropyLoss(),
        "optimiser": optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    }
    subset = True
    
    return run_epochs(
        model,
        get_train_cifar10(transform=transform, subset=subset),
        get_test_cifar10(transform=transform),
        hyperparams,
        start_epoch=0,
        n_epochs=1,
    )


In [9]:
study = optuna.create_study(direction="maximize") 
study.optimize(objective, n_trials=10)

# Print the best hyperparameters
print(f"Best trial: {study.best_trial.params}")

[I 2025-04-17 12:08:58,423] A new study created in memory with name: no-name-2cc3ba42-29ce-4b6d-aba7-dcef99a2fc6d


Epoch: 0


[W 2025-04-17 12:09:02,277] Trial 0 failed with parameters: {'userotation': True, 'usetranslation': True, 'useshearing': False, 'usehorizontal_flip': False, 'usevertical_flip': False, 'usecrop': False, 'usecolor_jitter': False, 'usekernel': True, 'userandom_erasing': False} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/homes/b24richa/Documents/EDL/.venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_278481/3588707239.py", line 28, in objective
    return run_epochs(
           ^^^^^^^^^^^
  File "/homes/b24richa/Documents/EDL/utils.py", line 83, in run_epochs
    train_acc, train_loss = train(
                            ^^^^^^
  File "/homes/b24richa/Documents/EDL/utils.py", line 49, in train
    train_loss += loss.item()
                  ^^^^^^^^^^^
KeyboardInterrupt
[W 2025-04-17 12:09:02,281] Trial 0 failed 

KeyboardInterrupt: 

In [1]:
import pickle

with open("best_transform.pkl", "rb") as f:
    best = pickle.load(f)

In [2]:
best

{'userotation': True,
 'usetranslation': False,
 'useshearing': False,
 'usehorizontal_flip': False,
 'usevertical_flip': False,
 'usecrop': False,
 'usecolor_jitter': False,
 'usekernel': False,
 'userandom_erasing': False}